In [57]:
import csv
import pandas as pd
#import pg8000
import sys
import os
import pathlib2
import lims_sql_query as lsq

# Mouse

In [108]:
home = pathlib2.Path.home()
full_path = home/"documents"/"github"/"SMART_goals"/"data"/"mouse_anno_table.csv"
full_path = str(full_path)
full_path
mshiny = pd.read_csv(full_path)

In [109]:
mshiny = mshiny[["anno.sample_id", "anno.res_index_label"]]
#, "anno.patch_date_label"
mshiny

,anno.sample_id,anno.res_index_label
0,P1S4_160318_069_A01,1.000000
1,P1S4_160407_108_A01,1.000000
2,P1S4_160407_109_A01,1.000000
3,P1S4_160407_112_A01,1.000000
4,P1S4_160407_113_A01,0.802691
5,P1S4_160407_114_A01,1.000000
6,P1S4_160408_121_A01,1.000000
7,P1S4_160408_123_A01,1.000000
8,P1S4_160408_124_A01,1.000000
9,P1S4_160408_125_A01,0.562351


In [110]:
#mshiny['anno.patch_date_label'] = pd.to_datetime(mshiny['anno.patch_date_label'], errors='coerce')

In [111]:
#mshiny = mshiny[mshiny['anno.patch_date_label'] >= '2018-01-01']
#mshiny

In [112]:
#getting all 63x_calls from LIMS
imgs_query = "Select cell.name, array_to_string(array_agg(DISTINCT tag.name), '_AND_') \
            FROM specimens cell JOIN ephys_roi_results err on err.id = cell.ephys_roi_result_id \
            JOIN specimen_tags_specimens sptagsp on sptagsp.specimen_id = cell.id \
            JOIN specimen_tags tag on tag.id = sptagsp.specimen_tag_id and tag.id in (602120185, 602122082) \
            GROUP BY cell.id \
            ORDER BY 1"
imgs_df = lsq.get_lims_dataframe(imgs_query)

In [113]:
lims_query1 = "WITH reporters AS (SELECT dg.donor_id, ARRAY_TO_STRING (ARRAY_AGG (DISTINCT g.name), ' ') AS reporters \
FROM donors_genotypes dg JOIN genotypes g ON dg.genotype_id = g.id \
JOIN genotype_types gt ON g.genotype_type_id = gt.id \
WHERE gt.name ILIKE 'reporter%' \
GROUP BY dg.donor_id ), \
drivers AS (SELECT dg.donor_id, ARRAY_TO_STRING (ARRAY_AGG (DISTINCT g.name), ' ') AS drivers \
FROM donors_genotypes dg \
JOIN genotypes g ON dg.genotype_id = g.id \
JOIN genotype_types gt ON g.genotype_type_id = gt.id \
WHERE gt.name ILIKE 'driver%' \
GROUP BY dg.donor_id) \
SELECT DISTINCT s.name AS cell_name, proj.name, proj.code, rpt.reporters, drv.drivers \
FROM specimens s JOIN projects proj ON s.project_id = proj.id \
LEFT JOIN reporters rpt ON s.donor_id = rpt.donor_id \
LEFT JOIN drivers drv ON s.donor_id = drv.donor_id \
WHERE proj.code = 'mIVSCC-MET' OR proj.code = 'T301' OR proj.code = 'T301x'"

lims_df1 = lsq.get_lims_dataframe(lims_query1)
lims_df1 = lims_df1.drop(['name', 'reporters'], axis=1)
lims_df1.tail()

,code,drivers,cell_name
84076,T301x,Htr3a-Cre_NO152,Htr3a-Cre_NO152;Ai14-200455.06.02
84077,T301x,Slc32a1-IRES-Cre,Slc32a1-IRES-Cre;Ai14-294509.02
84078,T301x,Gad2-IRES-Cre,Gad2-IRES-Cre;Ai14-272816.08.02
84079,T301x,Slc17a7-IRES2-Cre,Slc17a7-IRES2-Cre;Ai14-192350.01
84080,mIVSCC-MET,Scnn1a-Tg2-Cre,Scnn1a-Tg2-Cre;Ai14-358947.10


In [114]:
#getting all patched cell containers from LIMS
#lims_query = "SELECT specimens.name, specimens.patched_cell_container, err.recording_date \
#            FROM ephys_roi_results err JOIN specimens ON specimens.ephys_roi_result_id = err.id \
#            WHERE patched_cell_container != 'None'"

lims_query2 = "SELECT s.name, s.patched_cell_container \
            FROM specimens s \
            WHERE patched_cell_container != 'None'"
lims_df2 = lsq.get_lims_dataframe(lims_query2)
lims_df2.head()

,patched_cell_container,name
0,P8S4_171023_356_A01,Rorb-IRES2-Cre;Ai14-355893.04.02.01
1,P8S4_171025_353_A01,Rorb-IRES2-Cre;Ai14-355894.04.02.03
2,P9S4_171208_403_A01,Cux2-CreERT2;Ai14-362883.03.01.01
3,PAS4_171207_455_A01,Chrna2-Cre_OE25;Ai14-362810.05.01.03
4,P5S4_170406_204_A01,Gad2-IRES-Cre;Ai14-311014.05.01.01


In [115]:
lims_df = pd.merge(left = lims_df1, right = lims_df2, left_on = 'cell_name', right_on = 'name', how = 'right')
lims_df

,code,drivers,cell_name,patched_cell_container,name
0,T301x,Ndnf-IRES2-dgCre,Ndnf-IRES2-dgCre;Ai14-248399.05.01.04,P1S4_160603_168_A01,Ndnf-IRES2-dgCre;Ai14-248399.05.01.04
1,T301x,Nos1-CreERT2,Nos1-CreERT2;Ai14(BT)-240446.03.01.01,P1S4_160426_304_A01,Nos1-CreERT2;Ai14(BT)-240446.03.01.01
2,T301x,Pvalb-IRES-Cre,Pvalb-IRES-Cre;Ai14-316465.03.01.04,P6S4_170518_254_A01,Pvalb-IRES-Cre;Ai14-316465.03.01.04
3,T301x,Rbp4-Cre_KL100,Rbp4-Cre_KL100;Ai14-286970.05.01.05,P4S4_161216_155_A01,Rbp4-Cre_KL100;Ai14-286970.05.01.05
4,mIVSCC-MET,Th-Cre_FI172,Th-Cre_FI172;Ai14-363500.06.02.01,PAS4_180118_453_A01,Th-Cre_FI172;Ai14-363500.06.02.01
5,T301x,Sst-IRES-Cre,Sst-IRES-Cre;Ai14-331660.04.02.03,P8S4_170712_353_A01,Sst-IRES-Cre;Ai14-331660.04.02.03
6,mIVSCC-MET,Vip-IRES-Cre,Vip-IRES-Cre;Ai14-392332.03.01.02,P9S4_180529_407_A01,Vip-IRES-Cre;Ai14-392332.03.01.02
7,T301x,Gad2-IRES-Cre,Gad2-IRES-Cre;Ai14-272819.07.01.01,P1S4_160928_001_A01,Gad2-IRES-Cre;Ai14-272819.07.01.01
8,T301x,Ndnf-IRES2-dgCre,Ndnf-IRES2-dgCre;Ai14-303177.03.01.03,P1S4_170302_003_A01,Ndnf-IRES2-dgCre;Ai14-303177.03.01.03
9,mIVSCC-MET,None,Slc32a1-IRES-Cre;Ai14-384211.04.02.01,P8S4_180411_353_A01,Slc32a1-IRES-Cre;Ai14-384211.04.02.01


In [116]:
lims_df['recording_date'] = lims_df['patched_cell_container'].str[5:11]
lims_df

,code,drivers,cell_name,patched_cell_container,name,recording_date
0,T301x,Ndnf-IRES2-dgCre,Ndnf-IRES2-dgCre;Ai14-248399.05.01.04,P1S4_160603_168_A01,Ndnf-IRES2-dgCre;Ai14-248399.05.01.04,160603
1,T301x,Nos1-CreERT2,Nos1-CreERT2;Ai14(BT)-240446.03.01.01,P1S4_160426_304_A01,Nos1-CreERT2;Ai14(BT)-240446.03.01.01,160426
2,T301x,Pvalb-IRES-Cre,Pvalb-IRES-Cre;Ai14-316465.03.01.04,P6S4_170518_254_A01,Pvalb-IRES-Cre;Ai14-316465.03.01.04,170518
3,T301x,Rbp4-Cre_KL100,Rbp4-Cre_KL100;Ai14-286970.05.01.05,P4S4_161216_155_A01,Rbp4-Cre_KL100;Ai14-286970.05.01.05,161216
4,mIVSCC-MET,Th-Cre_FI172,Th-Cre_FI172;Ai14-363500.06.02.01,PAS4_180118_453_A01,Th-Cre_FI172;Ai14-363500.06.02.01,180118
5,T301x,Sst-IRES-Cre,Sst-IRES-Cre;Ai14-331660.04.02.03,P8S4_170712_353_A01,Sst-IRES-Cre;Ai14-331660.04.02.03,170712
6,mIVSCC-MET,Vip-IRES-Cre,Vip-IRES-Cre;Ai14-392332.03.01.02,P9S4_180529_407_A01,Vip-IRES-Cre;Ai14-392332.03.01.02,180529
7,T301x,Gad2-IRES-Cre,Gad2-IRES-Cre;Ai14-272819.07.01.01,P1S4_160928_001_A01,Gad2-IRES-Cre;Ai14-272819.07.01.01,160928
8,T301x,Ndnf-IRES2-dgCre,Ndnf-IRES2-dgCre;Ai14-303177.03.01.03,P1S4_170302_003_A01,Ndnf-IRES2-dgCre;Ai14-303177.03.01.03,170302
9,mIVSCC-MET,None,Slc32a1-IRES-Cre;Ai14-384211.04.02.01,P8S4_180411_353_A01,Slc32a1-IRES-Cre;Ai14-384211.04.02.01,180411


In [117]:
lims_df['recording_date'] = pd.to_datetime(lims_df['recording_date'], yearfirst=True, errors='coerce')

lims_df

,code,drivers,cell_name,patched_cell_container,name,recording_date
0,T301x,Ndnf-IRES2-dgCre,Ndnf-IRES2-dgCre;Ai14-248399.05.01.04,P1S4_160603_168_A01,Ndnf-IRES2-dgCre;Ai14-248399.05.01.04,2016-06-03
1,T301x,Nos1-CreERT2,Nos1-CreERT2;Ai14(BT)-240446.03.01.01,P1S4_160426_304_A01,Nos1-CreERT2;Ai14(BT)-240446.03.01.01,2016-04-26
2,T301x,Pvalb-IRES-Cre,Pvalb-IRES-Cre;Ai14-316465.03.01.04,P6S4_170518_254_A01,Pvalb-IRES-Cre;Ai14-316465.03.01.04,2017-05-18
3,T301x,Rbp4-Cre_KL100,Rbp4-Cre_KL100;Ai14-286970.05.01.05,P4S4_161216_155_A01,Rbp4-Cre_KL100;Ai14-286970.05.01.05,2016-12-16
4,mIVSCC-MET,Th-Cre_FI172,Th-Cre_FI172;Ai14-363500.06.02.01,PAS4_180118_453_A01,Th-Cre_FI172;Ai14-363500.06.02.01,2018-01-18
5,T301x,Sst-IRES-Cre,Sst-IRES-Cre;Ai14-331660.04.02.03,P8S4_170712_353_A01,Sst-IRES-Cre;Ai14-331660.04.02.03,2017-07-12
6,mIVSCC-MET,Vip-IRES-Cre,Vip-IRES-Cre;Ai14-392332.03.01.02,P9S4_180529_407_A01,Vip-IRES-Cre;Ai14-392332.03.01.02,2018-05-29
7,T301x,Gad2-IRES-Cre,Gad2-IRES-Cre;Ai14-272819.07.01.01,P1S4_160928_001_A01,Gad2-IRES-Cre;Ai14-272819.07.01.01,2016-09-28
8,T301x,Ndnf-IRES2-dgCre,Ndnf-IRES2-dgCre;Ai14-303177.03.01.03,P1S4_170302_003_A01,Ndnf-IRES2-dgCre;Ai14-303177.03.01.03,2017-03-02
9,mIVSCC-MET,None,Slc32a1-IRES-Cre;Ai14-384211.04.02.01,P8S4_180411_353_A01,Slc32a1-IRES-Cre;Ai14-384211.04.02.01,2018-04-11


In [118]:
#filter out dates prior to January 1, 2018
#lims_df = lims_df[lims_df['recording_date'] >= '2018-01-01 00:00:00']

lims_df = lims_df[lims_df['recording_date'] >= '2018-01-01']

lims_df

,code,drivers,cell_name,patched_cell_container,name,recording_date
4,mIVSCC-MET,Th-Cre_FI172,Th-Cre_FI172;Ai14-363500.06.02.01,PAS4_180118_453_A01,Th-Cre_FI172;Ai14-363500.06.02.01,2018-01-18
6,mIVSCC-MET,Vip-IRES-Cre,Vip-IRES-Cre;Ai14-392332.03.01.02,P9S4_180529_407_A01,Vip-IRES-Cre;Ai14-392332.03.01.02,2018-05-29
9,mIVSCC-MET,None,Slc32a1-IRES-Cre;Ai14-384211.04.02.01,P8S4_180411_353_A01,Slc32a1-IRES-Cre;Ai14-384211.04.02.01,2018-04-11
16,mIVSCC-MET,Slc17a8-IRES2-Cre,Slc17a8-IRES2-Cre;Ai14-390760.03.02.01,P8S4_180523_355_A01,Slc17a8-IRES2-Cre;Ai14-390760.03.02.01,2018-05-23
20,mIVSCC-MET,Sst-IRES-Cre,Sst-IRES-Cre;Ai14-376204.04.02.01,PBS4_180227_501_A01,Sst-IRES-Cre;Ai14-376204.04.02.01,2018-02-27
25,mIVSCC-MET,Vip-IRES-Cre,Vip-IRES-Cre;Ai14-382261.04.01.02,P1S4_180402_005_A01,Vip-IRES-Cre;Ai14-382261.04.01.02,2018-04-02
28,mIVSCC-MET,Ndnf-IRES2-dgCre Slc32a1-IRES2-FlpO,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-39146...,PAS4_180521_451_A01,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-39146...,2018-05-21
32,mIVSCC-MET,Th-P2A-FlpO,Th-P2A-FlpO;Ai65F-389710.04.02.01,P8S4_180504_354_A01,Th-P2A-FlpO;Ai65F-389710.04.02.01,2018-05-04
36,mIVSCC-MET,Sst-IRES-Cre,Sst-IRES-Cre;Ai14-372549.03.01.04,P1S4_180202_004_A01,Sst-IRES-Cre;Ai14-372549.03.01.04,2018-02-02
39,mIVSCC-MET,Ndnf-IRES2-dgCre Slc32a1-IRES2-FlpO,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-38181...,P1S4_180327_004_A01,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-38181...,2018-03-27


NOTE: A few recordings in LIMS are not updated - have no ephys sweeps, therefore no recording_date

In [119]:
#merge both lims queries together - probably should get these into a single query soon
lims = pd.merge(left = lims_df, right = imgs_df, left_on = 'name', right_on = 'name', how = 'left')
lims

,code,drivers,cell_name,patched_cell_container,name,recording_date,array_to_string
0,mIVSCC-MET,Th-Cre_FI172,Th-Cre_FI172;Ai14-363500.06.02.01,PAS4_180118_453_A01,Th-Cre_FI172;Ai14-363500.06.02.01,2018-01-18,NaN
1,mIVSCC-MET,Vip-IRES-Cre,Vip-IRES-Cre;Ai14-392332.03.01.02,P9S4_180529_407_A01,Vip-IRES-Cre;Ai14-392332.03.01.02,2018-05-29,NaN
2,mIVSCC-MET,None,Slc32a1-IRES-Cre;Ai14-384211.04.02.01,P8S4_180411_353_A01,Slc32a1-IRES-Cre;Ai14-384211.04.02.01,2018-04-11,NaN
3,mIVSCC-MET,Slc17a8-IRES2-Cre,Slc17a8-IRES2-Cre;Ai14-390760.03.02.01,P8S4_180523_355_A01,Slc17a8-IRES2-Cre;Ai14-390760.03.02.01,2018-05-23,NaN
4,mIVSCC-MET,Sst-IRES-Cre,Sst-IRES-Cre;Ai14-376204.04.02.01,PBS4_180227_501_A01,Sst-IRES-Cre;Ai14-376204.04.02.01,2018-02-27,63x no go
5,mIVSCC-MET,Vip-IRES-Cre,Vip-IRES-Cre;Ai14-382261.04.01.02,P1S4_180402_005_A01,Vip-IRES-Cre;Ai14-382261.04.01.02,2018-04-02,NaN
6,mIVSCC-MET,Ndnf-IRES2-dgCre Slc32a1-IRES2-FlpO,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-39146...,PAS4_180521_451_A01,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-39146...,2018-05-21,NaN
7,mIVSCC-MET,Th-P2A-FlpO,Th-P2A-FlpO;Ai65F-389710.04.02.01,P8S4_180504_354_A01,Th-P2A-FlpO;Ai65F-389710.04.02.01,2018-05-04,NaN
8,mIVSCC-MET,Sst-IRES-Cre,Sst-IRES-Cre;Ai14-372549.03.01.04,P1S4_180202_004_A01,Sst-IRES-Cre;Ai14-372549.03.01.04,2018-02-02,63x no go
9,mIVSCC-MET,Ndnf-IRES2-dgCre Slc32a1-IRES2-FlpO,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-38181...,P1S4_180327_004_A01,Ndnf-IRES2-dgCre;Slc32a1-IRES2-FlpO;Ai65-38181...,2018-03-27,NaN


In [120]:
#merge shiny and lims into single df
smart_data = pd.merge(left = mshiny, right = lims, left_on = 'anno.sample_id', right_on = 'patched_cell_container', how = 'right')
smart_data

,anno.sample_id,anno.res_index_label,code,drivers,cell_name,patched_cell_container,name,recording_date,array_to_string
0,P8S4_180102_351_A01,1.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.04.02.01,P8S4_180102_351_A01,Chat-IRES-Cre-neo;Ai14-365769.04.02.01,2018-01-02,63x go
1,P8S4_180102_352_A01,0.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.04.02.02,P8S4_180102_352_A01,Chat-IRES-Cre-neo;Ai14-365769.04.02.02,2018-01-02,NaN
2,P8S4_180102_353_A01,0.745543,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.05.02.01,P8S4_180102_353_A01,Chat-IRES-Cre-neo;Ai14-365769.05.02.01,2018-01-02,NaN
3,P9S4_180102_401_A01,0.880454,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366659.04.01.01,P9S4_180102_401_A01,Pdyn-T2A-CreERT2;Ai14-366659.04.01.01,2018-01-02,63x go
4,P9S4_180102_402_A01,1.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.03.01.01,P9S4_180102_402_A01,Chat-IRES-Cre-neo;Ai14-365769.03.01.01,2018-01-02,63x no go
5,P9S4_180102_403_A01,1.000000,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366659.05.01.01,P9S4_180102_403_A01,Pdyn-T2A-CreERT2;Ai14-366659.05.01.01,2018-01-02,63x go
6,P8S4_180103_351_A01,0.000000,mIVSCC-MET,Calb1-IRES2-Cre Sst-IRES-FlpO,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.01,P8S4_180103_351_A01,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.01,2018-01-03,NaN
7,P8S4_180103_352_A01,1.000000,mIVSCC-MET,Calb1-IRES2-Cre Sst-IRES-FlpO,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.02,P8S4_180103_352_A01,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.02,2018-01-03,63x no go
8,PAS4_180103_451_A01,0.880454,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366660.05.02.01,PAS4_180103_451_A01,Pdyn-T2A-CreERT2;Ai14-366660.05.02.01,2018-01-03,63x no go
9,PAS4_180103_452_A01,0.784887,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366660.04.02.01,PAS4_180103_452_A01,Pdyn-T2A-CreERT2;Ai14-366660.04.02.01,2018-01-03,63x go


In [121]:
#change header from "array_to_string" to "63x_cal"
#smart_data = smart_data.drop(columns=['anno.sample_id'])
smart_data = smart_data.drop('anno.sample_id', axis=1)
smart_data = smart_data.rename(index=str, columns={"array_to_string": "63x_call"})
smart_data

,anno.res_index_label,code,drivers,cell_name,patched_cell_container,name,recording_date,63x_call
0,1.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.04.02.01,P8S4_180102_351_A01,Chat-IRES-Cre-neo;Ai14-365769.04.02.01,2018-01-02,63x go
1,0.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.04.02.02,P8S4_180102_352_A01,Chat-IRES-Cre-neo;Ai14-365769.04.02.02,2018-01-02,NaN
2,0.745543,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.05.02.01,P8S4_180102_353_A01,Chat-IRES-Cre-neo;Ai14-365769.05.02.01,2018-01-02,NaN
3,0.880454,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366659.04.01.01,P9S4_180102_401_A01,Pdyn-T2A-CreERT2;Ai14-366659.04.01.01,2018-01-02,63x go
4,1.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.03.01.01,P9S4_180102_402_A01,Chat-IRES-Cre-neo;Ai14-365769.03.01.01,2018-01-02,63x no go
5,1.000000,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366659.05.01.01,P9S4_180102_403_A01,Pdyn-T2A-CreERT2;Ai14-366659.05.01.01,2018-01-02,63x go
6,0.000000,mIVSCC-MET,Calb1-IRES2-Cre Sst-IRES-FlpO,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.01,P8S4_180103_351_A01,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.01,2018-01-03,NaN
7,1.000000,mIVSCC-MET,Calb1-IRES2-Cre Sst-IRES-FlpO,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.02,P8S4_180103_352_A01,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.02,2018-01-03,63x no go
8,0.880454,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366660.05.02.01,PAS4_180103_451_A01,Pdyn-T2A-CreERT2;Ai14-366660.05.02.01,2018-01-03,63x no go
9,0.784887,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366660.04.02.01,PAS4_180103_452_A01,Pdyn-T2A-CreERT2;Ai14-366660.04.02.01,2018-01-03,63x go


In [122]:
#smart_data['recording_date'] = smart_data['recording_date'].dt.date
smart_data.sort_values(['recording_date', 'patched_cell_container'], ascending=[True, True])
mouse_smart_data = smart_data.set_index('recording_date')

mouse_smart_data

,anno.res_index_label,code,drivers,cell_name,patched_cell_container,name,63x_call
recording_date,,,,,,,
2018-01-02,1.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.04.02.01,P8S4_180102_351_A01,Chat-IRES-Cre-neo;Ai14-365769.04.02.01,63x go
2018-01-02,0.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.04.02.02,P8S4_180102_352_A01,Chat-IRES-Cre-neo;Ai14-365769.04.02.02,NaN
2018-01-02,0.745543,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.05.02.01,P8S4_180102_353_A01,Chat-IRES-Cre-neo;Ai14-365769.05.02.01,NaN
2018-01-02,0.880454,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366659.04.01.01,P9S4_180102_401_A01,Pdyn-T2A-CreERT2;Ai14-366659.04.01.01,63x go
2018-01-02,1.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.03.01.01,P9S4_180102_402_A01,Chat-IRES-Cre-neo;Ai14-365769.03.01.01,63x no go
2018-01-02,1.000000,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366659.05.01.01,P9S4_180102_403_A01,Pdyn-T2A-CreERT2;Ai14-366659.05.01.01,63x go
2018-01-03,0.000000,mIVSCC-MET,Calb1-IRES2-Cre Sst-IRES-FlpO,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.01,P8S4_180103_351_A01,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.01,NaN
2018-01-03,1.000000,mIVSCC-MET,Calb1-IRES2-Cre Sst-IRES-FlpO,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.02,P8S4_180103_352_A01,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.02,63x no go
2018-01-03,0.880454,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366660.05.02.01,PAS4_180103_451_A01,Pdyn-T2A-CreERT2;Ai14-366660.05.02.01,63x no go


In [123]:
mouse_smart_data = mouse_smart_data[~mouse_smart_data['name'].astype(str).str.startswith('H1')]

mouse_smart_data

,anno.res_index_label,code,drivers,cell_name,patched_cell_container,name,63x_call
recording_date,,,,,,,
2018-01-02,1.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.04.02.01,P8S4_180102_351_A01,Chat-IRES-Cre-neo;Ai14-365769.04.02.01,63x go
2018-01-02,0.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.04.02.02,P8S4_180102_352_A01,Chat-IRES-Cre-neo;Ai14-365769.04.02.02,NaN
2018-01-02,0.745543,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.05.02.01,P8S4_180102_353_A01,Chat-IRES-Cre-neo;Ai14-365769.05.02.01,NaN
2018-01-02,0.880454,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366659.04.01.01,P9S4_180102_401_A01,Pdyn-T2A-CreERT2;Ai14-366659.04.01.01,63x go
2018-01-02,1.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.03.01.01,P9S4_180102_402_A01,Chat-IRES-Cre-neo;Ai14-365769.03.01.01,63x no go
2018-01-02,1.000000,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366659.05.01.01,P9S4_180102_403_A01,Pdyn-T2A-CreERT2;Ai14-366659.05.01.01,63x go
2018-01-03,0.000000,mIVSCC-MET,Calb1-IRES2-Cre Sst-IRES-FlpO,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.01,P8S4_180103_351_A01,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.01,NaN
2018-01-03,1.000000,mIVSCC-MET,Calb1-IRES2-Cre Sst-IRES-FlpO,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.02,P8S4_180103_352_A01,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.02,63x no go
2018-01-03,0.880454,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366660.05.02.01,PAS4_180103_451_A01,Pdyn-T2A-CreERT2;Ai14-366660.05.02.01,63x no go


In [124]:
mouse_smart_data = mouse_smart_data[~mouse_smart_data['patched_cell_container'].astype(str).str.startswith('PX')]
mouse_smart_data = mouse_smart_data[~mouse_smart_data['patched_cell_container'].astype(str).str.startswith('PG')]
mouse_smart_data = mouse_smart_data[~mouse_smart_data['patched_cell_container'].astype(str).str.startswith('PH')]
mouse_smart_data = mouse_smart_data.drop('name', axis=1)
mouse_smart_data

,anno.res_index_label,code,drivers,cell_name,patched_cell_container,63x_call
recording_date,,,,,,
2018-01-02,1.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.04.02.01,P8S4_180102_351_A01,63x go
2018-01-02,0.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.04.02.02,P8S4_180102_352_A01,NaN
2018-01-02,0.745543,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.05.02.01,P8S4_180102_353_A01,NaN
2018-01-02,0.880454,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366659.04.01.01,P9S4_180102_401_A01,63x go
2018-01-02,1.000000,mIVSCC-MET,Chat-IRES-Cre-neo,Chat-IRES-Cre-neo;Ai14-365769.03.01.01,P9S4_180102_402_A01,63x no go
2018-01-02,1.000000,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366659.05.01.01,P9S4_180102_403_A01,63x go
2018-01-03,0.000000,mIVSCC-MET,Calb1-IRES2-Cre Sst-IRES-FlpO,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.01,P8S4_180103_351_A01,NaN
2018-01-03,1.000000,mIVSCC-MET,Calb1-IRES2-Cre Sst-IRES-FlpO,Calb2-IRES-Cre;Sst-IRES-FlpO;Ai65-366183.04.02.02,P8S4_180103_352_A01,63x no go
2018-01-03,0.880454,mIVSCC-MET,Pdyn-T2A-CreERT2,Pdyn-T2A-CreERT2;Ai14-366660.05.02.01,PAS4_180103_451_A01,63x no go


In [125]:
full_path2 = home/"documents"/"github"/"SMART_goals"/"data"/"mouse_smart_data.csv"
full_path2 = str(full_path2)
mouse_smart_data.to_csv(full_path2)

# Human

In [46]:
home = pathlib2.Path.home()
full_path = home/"documents"/"github"/"SMART_goals"/"data"/"human_anno_table.csv"
full_path = str(full_path)
full_path
hshiny = pd.read_csv(full_path)

In [47]:
hshiny = hshiny[["anno.sample_id", "anno.res_index_label", "anno.roi_label"]]
#, "anno.patch_date_label"
hshiny

,anno.sample_id,anno.res_index_label,anno.roi_label
0,P2S4_170323_053_A01,1.000000,Right frontal
1,P2S4_170323_054_A01,0.869212,Right frontal
2,P2S4_170323_055_A01,1.000000,Right frontal
3,P2S4_170323_056_A01,1.000000,Right frontal
4,P5S4_170323_201_A01,1.000000,Right frontal
5,P5S4_170323_205_A01,1.000000,Right frontal
6,P7S4_170323_301_A01,0.537958,Right frontal
7,P7S4_170323_303_A01,0.429759,Right frontal
8,P7S4_170323_305_A01,1.000000,Right frontal
9,P7S4_170323_308_A01,0.254718,Right frontal


In [48]:
hsmart_data = pd.merge(left = hshiny, right = lims, left_on = 'anno.sample_id', right_on = 'patched_cell_container', how = 'right')
hsmart_data

,anno.sample_id,anno.res_index_label,anno.roi_label,patched_cell_container,name,recording_date,array_to_string
0,P2S4_180109_054_A01,0.000000,FCx_L2,P2S4_180109_054_A01,H18.03.002.11.03.01,2018-01-09,63x go
1,P8S4_180109_357_A01,0.000000,FCx_L2,P8S4_180109_357_A01,H18.03.002.11.06.01,2018-01-09,63x no go
2,P8S4_180109_358_A01,1.000000,FCx_L2,P8S4_180109_358_A01,H18.03.002.11.06.02,2018-01-09,63x go
3,P8S4_180109_359_A01,0.000000,FCx_L5,P8S4_180109_359_A01,H18.03.002.11.06.03,2018-01-09,63x go
4,P8S4_180109_360_A01,0.504463,FCx_L5,P8S4_180109_360_A01,H18.03.002.11.06.04,2018-01-09,63x go
5,P9S4_180109_403_A01,0.000000,FCx_L2,P9S4_180109_403_A01,H18.03.002.11.04.01,2018-01-09,63x no go
6,P9S4_180109_404_A01,0.681172,FCx_L3,P9S4_180109_404_A01,H18.03.002.11.04.02,2018-01-09,63x no go
7,P9S4_180109_405_A01,0.000000,FCx_L2,P9S4_180109_405_A01,H18.03.002.11.04.03,2018-01-09,63x no go
8,P9S4_180109_406_A01,0.000000,FCx_L4,P9S4_180109_406_A01,H18.03.002.11.04.04,2018-01-09,63x no go
9,P9S4_180201_402_A01,0.537958,FCx_L5,P9S4_180201_402_A01,H18.06.001.11.09.01,2018-02-01,63x no go


In [49]:
#change header from "array_to_string" to "63x_cal"
#smart_data = smart_data.drop(columns=['anno.sample_id'])
hsmart_data = hsmart_data.drop('anno.sample_id', axis=1)
hsmart_data = hsmart_data.rename(index=str, columns={"array_to_string": "63x_call"})
hsmart_data

,anno.res_index_label,anno.roi_label,patched_cell_container,name,recording_date,63x_call
0,0.000000,FCx_L2,P2S4_180109_054_A01,H18.03.002.11.03.01,2018-01-09,63x go
1,0.000000,FCx_L2,P8S4_180109_357_A01,H18.03.002.11.06.01,2018-01-09,63x no go
2,1.000000,FCx_L2,P8S4_180109_358_A01,H18.03.002.11.06.02,2018-01-09,63x go
3,0.000000,FCx_L5,P8S4_180109_359_A01,H18.03.002.11.06.03,2018-01-09,63x go
4,0.504463,FCx_L5,P8S4_180109_360_A01,H18.03.002.11.06.04,2018-01-09,63x go
5,0.000000,FCx_L2,P9S4_180109_403_A01,H18.03.002.11.04.01,2018-01-09,63x no go
6,0.681172,FCx_L3,P9S4_180109_404_A01,H18.03.002.11.04.02,2018-01-09,63x no go
7,0.000000,FCx_L2,P9S4_180109_405_A01,H18.03.002.11.04.03,2018-01-09,63x no go
8,0.000000,FCx_L4,P9S4_180109_406_A01,H18.03.002.11.04.04,2018-01-09,63x no go
9,0.537958,FCx_L5,P9S4_180201_402_A01,H18.06.001.11.09.01,2018-02-01,63x no go


In [50]:
hsmart_data['layer'] = hsmart_data['anno.roi_label'].str[-1:]

In [52]:
#smart_data['recording date'] = smart_data['recording date'].dt.date
hsmart_data.sort_values(['recording_date', 'patched_cell_container'], ascending=[True, True])
human_smart_data = hsmart_data.set_index('recording_date')
human_smart_data = human_smart_data.drop('anno.roi_label', axis=1)
human_smart_data

,anno.res_index_label,patched_cell_container,name,63x_call,layer
recording_date,,,,,
2018-01-09,0.000000,P2S4_180109_054_A01,H18.03.002.11.03.01,63x go,2
2018-01-09,0.000000,P8S4_180109_357_A01,H18.03.002.11.06.01,63x no go,2
2018-01-09,1.000000,P8S4_180109_358_A01,H18.03.002.11.06.02,63x go,2
2018-01-09,0.000000,P8S4_180109_359_A01,H18.03.002.11.06.03,63x go,5
2018-01-09,0.504463,P8S4_180109_360_A01,H18.03.002.11.06.04,63x go,5
2018-01-09,0.000000,P9S4_180109_403_A01,H18.03.002.11.04.01,63x no go,2
2018-01-09,0.681172,P9S4_180109_404_A01,H18.03.002.11.04.02,63x no go,3
2018-01-09,0.000000,P9S4_180109_405_A01,H18.03.002.11.04.03,63x no go,2
2018-01-09,0.000000,P9S4_180109_406_A01,H18.03.002.11.04.04,63x no go,4


In [53]:
human_smart_data = human_smart_data[human_smart_data['name'].astype(str).str.startswith('H1')]

human_smart_data

,anno.res_index_label,patched_cell_container,name,63x_call,layer
recording_date,,,,,
2018-01-09,0.000000,P2S4_180109_054_A01,H18.03.002.11.03.01,63x go,2
2018-01-09,0.000000,P8S4_180109_357_A01,H18.03.002.11.06.01,63x no go,2
2018-01-09,1.000000,P8S4_180109_358_A01,H18.03.002.11.06.02,63x go,2
2018-01-09,0.000000,P8S4_180109_359_A01,H18.03.002.11.06.03,63x go,5
2018-01-09,0.504463,P8S4_180109_360_A01,H18.03.002.11.06.04,63x go,5
2018-01-09,0.000000,P9S4_180109_403_A01,H18.03.002.11.04.01,63x no go,2
2018-01-09,0.681172,P9S4_180109_404_A01,H18.03.002.11.04.02,63x no go,3
2018-01-09,0.000000,P9S4_180109_405_A01,H18.03.002.11.04.03,63x no go,2
2018-01-09,0.000000,P9S4_180109_406_A01,H18.03.002.11.04.04,63x no go,4


In [54]:
human_smart_data = human_smart_data[~human_smart_data['patched_cell_container'].astype(str).str.startswith('PX')]
human_smart_data = human_smart_data[~human_smart_data['patched_cell_container'].astype(str).str.startswith('PG')]
human_smart_data = human_smart_data[~human_smart_data['patched_cell_container'].astype(str).str.startswith('PH')]

human_smart_data

,anno.res_index_label,patched_cell_container,name,63x_call,layer
recording_date,,,,,
2018-01-09,0.000000,P2S4_180109_054_A01,H18.03.002.11.03.01,63x go,2
2018-01-09,0.000000,P8S4_180109_357_A01,H18.03.002.11.06.01,63x no go,2
2018-01-09,1.000000,P8S4_180109_358_A01,H18.03.002.11.06.02,63x go,2
2018-01-09,0.000000,P8S4_180109_359_A01,H18.03.002.11.06.03,63x go,5
2018-01-09,0.504463,P8S4_180109_360_A01,H18.03.002.11.06.04,63x go,5
2018-01-09,0.000000,P9S4_180109_403_A01,H18.03.002.11.04.01,63x no go,2
2018-01-09,0.681172,P9S4_180109_404_A01,H18.03.002.11.04.02,63x no go,3
2018-01-09,0.000000,P9S4_180109_405_A01,H18.03.002.11.04.03,63x no go,2
2018-01-09,0.000000,P9S4_180109_406_A01,H18.03.002.11.04.04,63x no go,4


In [55]:
full_path2 = home/"documents"/"github"/"SMART_goals"/"data"/"human_smart_data.csv"
full_path2 = str(full_path2)
#human_smart_data.to_csv(full_path2)